# Demonstration of self-calibration of defocus position with unknown samples
Cao, Ruiming, et al. "Self-calibrated 3D differential phase contrast microscopy with optimized illumination." Biomedical Optics Express 13.3 (2022): 1671-1684.

[Paper link](https://opg.optica.org/boe/fulltext.cfm?uri=boe-13-3-1671&id=469840)

Code author: Ruiming Cao, rcao@berkeley.edu

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import os, random
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

from algorithm_throughfocus import ThroughFocusSolver

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


# Set up parameters and load experimental dataset

In [26]:
wavelength     = 0.525
mag            = 40
na             = 0.55
na_in          = 0.0
pixel_size_cam = 6.5
pixel_size     = pixel_size_cam/mag #in micron
RI_medium      =   1.58             #background refractive index
pixel_size_z   =  1.0   #in micron
rotation       = [180, 0, 270, 90]
fx_illu = 0.361 / wavelength
fy_illu = 0.0

with np.load('single_led_im_stack.npz') as data:
    # off axis led is used by default in this code. 
    # on axis led can be used by setting fx_illu = 0.0
    im_stack_offaxis_ = data['im_stack_offaxis_']
    im_stack_onaxis_ = data['im_stack_onaxis_']


# Generate a random z defocus trajectory

In [ ]:
img_ind_z = [0]
for i in range(99):
    img_ind_z.append(random.randint(1,im_stack_offaxis_.shape[0]-1))

img_ind_z = sorted(img_ind_z)

z_dist = [z * pixel_size_z for z in img_ind_z]

print(img_ind_z)
img_sel_onaxis_ = im_stack_onaxis_[img_ind_z]
img_sel_offaxis_ = im_stack_offaxis_[img_ind_z]
print(len(set(img_ind_z)))
z_dist = [z - z_dist[0] for z in z_dist]

[0, 1, 2, 2, 3, 4, 6, 6, 7, 8, 8, 11, 14, 17, 19, 21, 21, 21, 21, 21, 27, 30, 33, 33, 34, 35, 35, 36, 37, 37, 38, 40, 40, 41, 42, 42, 44, 46, 47, 47, 49, 53, 54, 58, 58, 61, 62, 62, 62, 63, 64, 65, 65, 65, 66, 72, 74, 75, 75, 78, 82, 84, 84, 87, 87, 88, 91, 94, 100, 100, 102, 103, 104, 104, 105, 106, 107, 107, 109, 109, 109, 111, 111, 112, 116, 117, 119, 124, 126, 127, 128, 128, 129, 129, 131, 131, 131, 131, 132, 137]
(100, 300, 300)
68


## Visualize Intensity

In [27]:
f, ax  = plt.subplots(3, 4, sharex=True, sharey=True, figsize=(9, 4))
frames = []
for plot_index in range(12):
    plot_row = plot_index // 4
    plot_col = plot_index % 4
    ax[plot_row,plot_col].imshow(img_sel_offaxis_[plot_index*8], clim=(0.5, 1.5), cmap="gray",)
    ax[plot_row,plot_col].axis("off")

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Init solver

In [ ]:
z_dist_ = list(np.array(z_dist).astype(np.float32)-60)
fx_illu = 0.361 / wavelength
pad = 2
solver = ThroughFocusSolver(img_sel_offaxis_, pixel_size, wavelength,NA=na,z_planes=z_dist_, fx_illu=-fx_illu, fy_illu=-fy_illu,pad=(True,pad))

[-60.0, -58.0, -58.0, -58.0, -57.0, -55.0, -52.0, -51.0, -48.0, -48.0, -47.0, -45.0, -44.0, -42.0, -41.0, -39.0, -39.0, -38.0, -38.0, -37.0, -36.0, -35.0, -35.0, -33.0, -30.0, -30.0, -24.0, -23.0, -23.0, -21.0, -17.0, -16.0, -15.0, -8.0, -8.0, -8.0, -7.0, -6.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 4.0, 5.0, 5.0, 5.0, 6.0, 7.0, 9.0, 11.0, 11.0, 14.0, 15.0, 15.0, 16.0, 17.0, 20.0, 20.0, 23.0, 27.0, 28.0, 29.0, 30.0, 30.0, 30.0, 31.0, 32.0, 35.0, 36.0, 37.0, 42.0, 42.0, 42.0, 43.0, 44.0, 44.0, 47.0, 50.0, 50.0, 50.0, 53.0, 54.0, 54.0, 54.0, 55.0, 57.0, 58.0, 63.0, 64.0, 64.0, 65.0, 68.0, 71.0, 72.0, 73.0, 74.0, 76.0]


## Run joint optimization of defocus positions and light field

In [ ]:
start_pos = -70
z_dist_init = [1.0*i*140/(img_sel_onaxis_.shape[0])+start_pos for i in range(img_sel_onaxis_.shape[0])]
print('inital linear guess of z positions:')
print(z_dist_init)

s = 3  # skip every s frames for faster computation. s=1 for all frames
pad = 2

solver = ThroughFocusSolver(img_sel_offaxis_[::s], pixel_size, wavelength, na, z_dist_init[::s], fx_illu=-fx_illu, fy_illu=-fy_illu,pad=(True,pad))
x_log, z_log, _, err_x_log, err_z_log, _ = solver.joint_solve_xz(z_dist_init[::s], 
                                                         lr_x=1e-2, lr_z=1e-2,iterations=50)


## Visualize z 

In [ ]:
f, ax  = plt.subplots(1, 3, figsize=(10, 4))
frame = []
ax[0].plot(np.arange(len(z_log[0])), [z-70 for z in z_dist[::s]], 'ro', label='groundtruth')
frame.append(ax[0].plot(np.arange(len(z_log[0])), z_log[0],'v', label='estmiated'))
frame.append(ax[1].plot(np.arange(len(z_log[0])-1), np.abs(np.array(z_dist[1+s-1::s])-np.array(z_dist[:-1-s+1:s]) - (z_log[0][1:]-z_log[0][:-1])), 'o'))
frame.append(ax[2].plot(np.arange(len(err_x_log)-1), err_x_log[1:], 'o'))
# frame.append(ax[3].plot(np.arange(len(err_x_log)), err_x_log))
ax[0].axis([-1.5, len(z_log[0])+1.5, np.min(np.array(z_log))-15, np.max(np.array(z_log))+15])
ax[1].axis([-1.5, len(z_log[0])+1.5, -1, 25])
ax[2].axis([-2, len(err_x_log)+2, np.min(np.array(err_x_log))-15, np.max(np.array(err_x_log))+15])

ax[0].set_title('defocus position')
ax[1].set_title('defocus position error')
ax[2].set_title('loss')
ax[0].set_xlabel('time')
ax[0].set_ylabel('z, µm')
ax[1].set_xlabel('time')
ax[1].set_ylabel('abs(actual defocus - predicted defocus), µm')
ax[2].set_ylabel('loss')
ax[2].set_xlabel('iteration')
ax[0].legend()
plt.tight_layout()

def updateFrames(iteration):
    frame[0][0].set_ydata(z_log[iteration])
    ax[0].set_title('defocus position, iter={:d}'.format(iteration))
    frame[1][0].set_ydata(np.abs(np.array(z_dist[1+s-1::s])-np.array(z_dist[:-1-s+1:s]) - (z_log[iteration][1:]-z_log[iteration][:-1])))
#     frame[2][0].set_ydata(err_log[z_index])
    frame[2][0].set_data(np.arange(iteration), err_x_log[:iteration])

interact(updateFrames, iteration=IntSlider(min=0,max=len(z_log)-1,step=1,value=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='iteration', max=50), Output()), _dom_classes=('widget-in…

<function __main__.updateFrames>